# Processamento de CSVs e cálculo de média

Este notebook percorre todos os arquivos CSV com sufixo `_DataN.csv` em um diretório,
agrupa-os pelo prefixo antes de `_DataN.csv` e calcula a média linha a linha para cada grupo.
Gera um arquivo `<prefix>_avg.csv` para cada conjunto de réplicas.

In [4]:
import os
import re
import pandas as pd
from glob import glob


In [6]:
# --- caminho da pasta onde estão seus CSVs ---
input_dir = r"C:\\Supaero\\Stage\\Codigo3\\Analysis\\Data\\Combination"
# encontra todos os arquivos .csv
csv_paths = glob(os.path.join(input_dir, "*.csv"))

In [7]:
# agrupa por prefixo "<algo>_Data" (por ex. "A4m_B50u_D5m")
groups = {}
pattern = re.compile(r'^(.+)_Data\d+\.csv$', re.IGNORECASE)
for path in csv_paths:
    fn = os.path.basename(path)
    m = pattern.match(fn)
    if m:
        prefix = m.group(1)
        groups.setdefault(prefix, []).append(path)

# para cada prefixo, calcula a média e salva "<prefix>_avg.csv"
for prefix, files in groups.items():
    # lê todos os Data*.csv daquele grupo, na ordem Data0, Data1, ...
    files = sorted(files, key=lambda x: int(re.search(r'_Data(\d+)\.csv$', x).group(1)))
    dfs   = [pd.read_csv(f) for f in files]
    # garante que todos têm mesmas colunas e índices
    avg_df = sum(dfs) / len(dfs)
    # salva sem incluir índice extra
    out_path = os.path.join(input_dir, f"{prefix}_avg.csv")
    avg_df.to_csv(out_path, index=False)
    print(f"Gerado: {out_path}")